In [1]:
import os

CONSUMER_KEY = os.environ.get('X_ACESS_KEY')
CONSUMER_SECRET = os.environ.get('X_SECRET_KEY')

In [2]:
import webbrowser
from twython import Twython

In [3]:
# Get a temporary client to retrieve the authentication URL
temp_client = Twython(CONSUMER_KEY, CONSUMER_SECRET)
temp_creds = temp_client.get_authentication_tokens()
url = temp_creds['auth_url']

In [4]:
# Now visit that URL to authorize the application and get a PIN
print("Go visit this URL and get the PIN: " + url)
webbrowser.open(url)
PIN = input("Enter the PIN: ")

Go visit this URL and get the PIN: https://api.twitter.com/oauth/authenticate?oauth_token=QA0EwgAAAAABxSQRAAABk4AakPM


Gtk-Message: 23:45:54.598: Not loading module "atk-bridge": The functionality is provided by GTK natively. Please try to not load it.
[116193, Main Thread] WARNING: GTK+ module /snap/firefox/5361/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same process is not supported.: 'glib warning', file /build/firefox/parts/firefox/build/toolkit/xre/nsSigHandlers.cpp:201

(firefox:116193): Gtk-WARNING **: 23:45:54.668: GTK+ module /snap/firefox/5361/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same process is not supported.
Gtk-Message: 23:45:54.668: Failed to load module "canberra-gtk-module"
[116193, Main Thread] WARNING: GTK+ module /snap/firefox/5361/gnome-platform/usr/lib/gtk-2.0/modules/libcanberra-gtk-module.so cannot be loaded.
GTK+ 2.x symbols detected. Using GTK+ 2.x and GTK+ 3 in the same proc

In [5]:
# Now we use that PIN to get the actual tokens
auth_client = Twython(CONSUMER_KEY, CONSUMER_SECRET,
                      temp_creds['oauth_token'],
                      temp_creds['oauth_token_secret'])
final_step = auth_client.get_authorized_tokens(PIN)

In [ ]:
ACCESS_TOKEN = final_step['oauth_token']
ACCESS_TOKEN_SECRET = final_step['oauth_token_secret']

In [7]:
# And get a new Twython instance using them
twitter = Twython(CONSUMER_KEY, CONSUMER_SECRET,
                  ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [8]:
# Search for tweets containing the phrase "data science"
for status in twitter.search(q='"data science"')["statuses"]:
    user = status["user"]["screen_name"].encode('utf-8')
    text = status["text"].encode('utf-8')
    print(user, ":", text)

TwythonError: Twitter API returned a 403 (Forbidden), You currently have access to a subset of X API V2 endpoints and limited v1.1 endpoints (e.g. media post, oauth) only. If you need access to this endpoint, you may need a different access level. You can learn more here: https://developer.x.com/en/portal/product

In [ ]:
from twython import TwythonStreamer

# Appending data to a global variable is pretty poor form
# but it makes the example much simpler
tweets = []


class MyStreamer(TwythonStreamer):
    def on_success(self, data):
        # Only want to collect English-language tweets
        if data['lang'] == 'en':
            tweets.append(data)
            print("received tweet #", len(tweets))

        # Stop when we've collected enough
        if len(tweets) >= 1000:
            self.disconnect()

    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

In [ ]:
stream = MyStreamer(CONSUMER_KEY, CONSUMER_SECRET,
                    ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
# starts consuming public statuses that contain the keyword 'data'
stream.statuses.filter(track='data')

In [ ]:
from collections import Counter

top_hashtags = Counter(hashtag['text'].lower()
                       for tweet in tweets
                       for hashtag in tweet["entities"]["hashtags"])

print(top_hashtags.most_common(5))